In [1]:
import boto3

clientec2 = boto3.client('ec2')
resp = clientec2.run_instances(ImageId = 'ami-0a0ad6b70e61be944', InstanceType = 't2.micro', MinCount = 1, MaxCount = 1, KeyName = 'momo_keypair', SecurityGroupIds=['sg-0a36ff444c898b8d5'], SecurityGroups=['launch-wizard-3'])
instance_ids = []
for instance in resp['Instances']:
    instance_id = instance['InstanceId']
    instance_ids.append(instance_id)
    print(instance_ids)


ec2 = boto3.resource('ec2', region_name='us-east-2')
instance = ec2.Instance(id=instance_id)
instance.wait_until_running()
current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
ip_address = current_instance[0].public_ip_address




['i-09160f37d53e3a02e']


In [3]:
print(ip_address)

18.216.99.209


In [4]:
import paramiko
import botocore
import time

In [5]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [6]:
def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    privkey = paramiko.RSAKey.from_private_key_file(
        '/home/aymen/Downloads/momo_keypair.pem')
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ec2-user', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)

In [87]:
ssh_connect_with_retry(ssh, ip_address, 0)

SSH into the instance: 18.191.51.69
[Errno 110] Connection timed out
Retrying SSH connection to 18.191.51.69
SSH into the instance: 18.191.51.69
[Errno 110] Connection timed out
Retrying SSH connection to 18.191.51.69
SSH into the instance: 18.191.51.69
[Errno 110] Connection timed out
Retrying SSH connection to 18.191.51.69
SSH into the instance: 18.191.51.69
[Errno 110] Connection timed out
Retrying SSH connection to 18.191.51.69


In [1]:
stdin, stdout, stderr = ssh.exec_command("echo 'Hello World!'")
print('stdout:', stdout.read())
print('stderr:', stderr.read())

NameError: name 'ssh' is not defined

In [41]:
print(instance_ids)
commands = ['echo "hello world"']
clientssm = boto3.client('ssm')
respssm = clientssm.send_command(DocumentName = "AWS-RunShellScript", Parameters = {'commands': commands}, InstanceIds = instance_ids)

['i-09634fa2ea3a29e2a']


InvalidInstanceId: An error occurred (InvalidInstanceId) when calling the SendCommand operation: 

In [20]:
ec2 = boto3.resource('ec2')
instance = ec2.Instance(instance_id)
response = instance.stop()

In [50]:
import os

stream = os.popen('ls')
output = stream.read()
output

In [57]:
import subprocess
import shlex
subprocess.call(shlex.split('./test.sh param1 param2'))

0